In [1]:
import json

In [2]:
with open("gamemaster.json","r",encoding="utf-8") as file:
    gamemaster=json.load(file)
with open("dict_cp_multiplier.json","r",encoding="utf-8") as file:
    dic=json.load(file)
with open("typechart.json","r",encoding="utf-8") as file:
    typechart=json.load(file)
    
pokemons = gamemaster['pokemon']
moves = gamemaster['moves']

In [10]:
pokemons

[{'dex': 1,
  'speciesName': 'Bulbasaur',
  'speciesId': 'bulbasaur',
  'baseStats': {'atk': 118, 'def': 111, 'hp': 128},
  'types': ['grass', 'poison'],
  'fastMoves': ['TACKLE', 'VINE_WHIP'],
  'chargedMoves': ['POWER_WHIP', 'SEED_BOMB', 'SLUDGE_BOMB'],
  'tags': ['starter'],
  'defaultIVs': {'cp1500': [40, 15, 15, 15], 'cp2500': [40, 15, 15, 15]},
  'shadow': True,
  'level25CP': 627},
 {'dex': 2,
  'speciesName': 'Ivysaur',
  'speciesId': 'ivysaur',
  'baseStats': {'atk': 151, 'def': 143, 'hp': 155},
  'types': ['grass', 'poison'],
  'fastMoves': ['RAZOR_LEAF', 'VINE_WHIP'],
  'chargedMoves': ['POWER_WHIP', 'SLUDGE_BOMB', 'SOLAR_BEAM'],
  'tags': ['starter'],
  'defaultIVs': {'cp1500': [36, 6, 13, 12], 'cp2500': [40, 15, 15, 15]},
  'shadow': True,
  'level25CP': 1002},
 {'dex': 3,
  'speciesName': 'Venusaur',
  'speciesId': 'venusaur',
  'baseStats': {'atk': 198, 'def': 189, 'hp': 190},
  'types': ['grass', 'poison'],
  'fastMoves': ['RAZOR_LEAF', 'VINE_WHIP'],
  'chargedMoves': [

In [13]:
class Pokemon:
    def __init__(self, data):
        self.name = data['speciesName']
        self.baseStats = data['baseStats']
        self.types = data['types']
        self.fastMoves = data['fastMoves']
        self.chargedMoves = data['chargedMoves']

class Player:
    def __init__(self, name):
        self.name = name
        self.pokemon = None
        
    def choose_pokemon(self):
        POKEMON_NAME = input("請輸入您想要選擇的寶可夢: ")
        self.pokemon = Pokemon([item for item in pokemons if item['speciesName'] == POKEMON_NAME][0])
        
class Move:
    def __init__(self, data):
        self.name = data['name']
        self.types = data['type']
        self.power = data['power']
        self.energy = data['energy']
        self.energyGain = data['energyGain']
        self.cooldown = data['cooldown']
        
class Combat:
    def __init__(self, player1, player2):
        self.player = [player1, player2]
        self.stage = 0
        self.dmg_mult = [0, 0]
        self.selected_fast_move = [None, None]
        self.selected_charged_move = [None, None]
        self.energy = [0, 0]
        self.fastMove_cd = [0, 0]
        self.chargedMove_cd = [0, 0]
        self.hp = [player1.pokemon.baseStats['hp'], player2.pokemon.baseStats['hp']]
        
        
    def info(self):
        """檢查遊戲狀態"""
        for i in range(2):
            print("玩家{}: {}\nPokemon: {}\n傷害係數: {}\n選擇的小招: {}\n選擇的大招: {}\n".format(i + 1, 
                  self.player[i].name, self.player[i].pokemon.name, self.dmg_mult[i], 
                  self.selected_fast_move[i].name, self.selected_charged_move[i]))

    def check_type(self,typechart):
        """檢查雙方的屬性相剋"""
        self.dmg_mult = [1, 1]
        for p1_type in self.player[0].pokemon.types:
            if p1_type != 'none':
                for p2_type in self.player[1].pokemon.types:
                    if p2_type != 'none':
                        if p1_type in typechart[p2_type]['traits']['weaknesses']:
                            self.dmg_mult[0] = self.dmg_mult[0] * 1.6
                            print('p1 剋制 p2')
                        if p1_type in typechart[p2_type]['traits']['resistances']:
                            self.dmg_mult[0] = self.dmg_mult[0] * 0.625
                            print('p2 抵抗 p1')
                        if p1_type in typechart[p2_type]['traits']['immunities']:
                            self.dmg_mult[0] = 0
                            print('p2 免疫 p1')
        
        for p2_type in self.player[1].pokemon.types:
            if p2_type != 'none':
                for p1_type in self.player[0].pokemon.types:
                    if p1_type != 'none':
                        if p2_type in typechart[p1_type]['traits']['weaknesses']:
                            self.dmg_mult[1] = self.dmg_mult[1] * 1.6
                            print('p2 剋制 p1')
                        if p2_type in typechart[p1_type]['traits']['resistances']:
                            self.dmg_mult[1] = self.dmg_mult[1] * 0.625
                            print('p1 抵抗 p2')
                        if p2_type in typechart[p1_type]['traits']['immunities']:
                            self.dmg_mult[1] = 0
                            print('p1 免疫 p2')
                            
        print("玩家1的傷害係數為 {}, 玩家2的傷害係數為 {}".format(self.dmg_mult[0],self.dmg_mult[1]))
        
    def select_fast_move(self, player_num):
        """玩家選擇小招"""
        print("{} 能出的小招有: {}".format(self.player[player_num - 1].pokemon.name, 
                                      self.player[player_num - 1].pokemon.fastMoves))
        while True:
            selected_fast_move = input("請輸入這局遊戲要使用的小招: ")
            if selected_fast_move in self.player[player_num - 1].pokemon.fastMoves:
               break 
            print("錯誤: 請輸入 {} 所擁有的小招".format(self.player[player_num - 1].pokemon.name))
            
        self.selected_fast_move[player_num - 1] = Move([move for move in moves if move['moveId'] == selected_fast_move][0])
        
            
    def select_charged_move(self, player_num):
        """玩家選擇大招"""
        print("{} 能出的大招有: {}".format(self.player[player_num - 1].pokemon.name, 
                                      self.player[player_num - 1].pokemon.chargedMoves))
        while True:
            selected_charged_move = input("請輸入這局遊戲要使用的大招: ")
            if selected_charged_move in self.player[player_num - 1].pokemon.chargedMoves:
               break 
            print("錯誤: 請輸入 {} 所擁有的大招".format(self.player[player_num - 1].pokemon.name))
            
        self.selected_charged_move[player_num - 1] = Move([move for move in moves if move['moveId'] == selected_charged_move][0])    
        
    def attack(self, attacker, move):
        atkplus = 1
        if move.types in self.player[attacker].pokemon.types:
            atkplus = 1.25
        atk_divide_deff = self.player[attacker].pokemon.baseStats['atk'] / self.player[1 - attacker].pokemon.baseStats['def']
        damage = 1 + round(0.5 * move.power * atk_divide_deff * atkplus * self.dmg_mult[attacker])
        self.hp[1 - attacker] = self.hp[1 - attacker] - damage
        if move.energy == 0:
            self.fastMove_cd[attacker] = self.fastMove_cd[attacker] + move.cooldown
            self.energy[attacker] = self.energy[attacker] + move.energyGain
        else:
            self.chargedMove_cd[attacker] = self.chargedMove_cd[attacker] + move.cooldown
            self.energy[attacker] = self.energy[attacker] - move.energy
    
    def start(self):
        while self.hp[0] > 0 and self.hp[1] > 0:
            self.stage = self.stage + 1
            print("Round {}".format(self.stage))
            for player_num in range(2):
                if self.energy[player_num] >= self.selected_charged_move[player_num].energy and self.chargedMove_cd[player_num] == 0:
                    self.attack(player_num, self.selected_charged_move[player_num])
                    print("Player {} 的 {} 使出大招".format(player_num + 1, self.player[player_num].pokemon.name))
                elif self.fastMove_cd[player_num] == 0:
                    self.attack(player_num, self.selected_fast_move[player_num])
                    print("Player {} 的 {} 使出小招".format(player_num + 1, self.player[player_num].pokemon.name))
                else:
                    print("Player {} 的 {} 招式冷卻中".format(player_num + 1, self.player[player_num].pokemon.name))
            
            print("\n")
            
            for player_num in range(2):
                self.fastMove_cd[player_num] = max(self.fastMove_cd[player_num] - 1000, 0)
                self.chargedMove_cd[player_num] = max(self.chargedMove_cd[player_num] - 1000, 0)
    
            if self.hp[0] <= 0 and self.hp[1] > 0:
                print("p2 win")
            elif self.hp[1] <= 0 and self.hp[0] > 0:
                print("p1 win")
            elif self.hp[1] <= 0 and self.hp[0] <= 0:
                print("draw")
            else:
                for i in range(2):
                    print("Player {} 的 {}:\nHP: {}/{}\nenergy: {}\n小招冷卻: {}\n大招冷卻: {}\n".format(i + 1, self.player[i].pokemon.name, 
                                                                            self.hp[i], self.player[i].pokemon.baseStats['hp'],
                                                                            self.energy[i], self.fastMove_cd[i], self.chargedMove_cd[i]))
            
        
        
        
        
        

In [4]:
p1 = Player('YiWei')
p2 = Player('Mingtzung')

In [5]:
p1.choose_pokemon()
p2.choose_pokemon()

請輸入您想要選擇的寶可夢: Pidgey
請輸入您想要選擇的寶可夢: Pikachu


In [6]:
combat = Combat(p1,p2)

In [7]:
combat.check_type(typechart)

p2 抵抗 p1
p2 剋制 p1
玩家1的傷害係數為 0.625, 玩家2的傷害係數為 1.6


In [8]:
combat.select_fast_move(1)
combat.select_fast_move(2)
combat.select_charged_move(1)
combat.select_charged_move(2)

Pidgey 能出的小招有: ['QUICK_ATTACK', 'TACKLE']
請輸入這局遊戲要使用的小招: QUICK_ATTACK
Pikachu 能出的小招有: ['PRESENT', 'QUICK_ATTACK', 'THUNDER_SHOCK']
請輸入這局遊戲要使用的小招: QUICK_ATTACK
Pidgey 能出的大招有: ['AERIAL_ACE', 'AIR_CUTTER', 'TWISTER']
請輸入這局遊戲要使用的大招: TWISTER
Pikachu 能出的大招有: ['DISCHARGE', 'SURF', 'THUNDER', 'THUNDERBOLT', 'WILD_CHARGE']
請輸入這局遊戲要使用的大招: SURF


In [9]:
combat.start()

Round 1
Player 1 的 Pidgey 使出小招
Player 2 的 Pikachu 使出小招


Player 1 的 Pidgey:
HP: 113/120
energy: 7
小招冷卻: 0
大招冷卻: 0

Player 2 的 Pikachu:
HP: 108/111
energy: 7
小招冷卻: 0
大招冷卻: 0

Round 2
Player 1 的 Pidgey 使出小招
Player 2 的 Pikachu 使出小招


Player 1 的 Pidgey:
HP: 106/120
energy: 14
小招冷卻: 0
大招冷卻: 0

Player 2 的 Pikachu:
HP: 105/111
energy: 14
小招冷卻: 0
大招冷卻: 0

Round 3
Player 1 的 Pidgey 使出小招
Player 2 的 Pikachu 使出小招


Player 1 的 Pidgey:
HP: 99/120
energy: 21
小招冷卻: 0
大招冷卻: 0

Player 2 的 Pikachu:
HP: 102/111
energy: 21
小招冷卻: 0
大招冷卻: 0

Round 4
Player 1 的 Pidgey 使出小招
Player 2 的 Pikachu 使出小招


Player 1 的 Pidgey:
HP: 92/120
energy: 28
小招冷卻: 0
大招冷卻: 0

Player 2 的 Pikachu:
HP: 99/111
energy: 28
小招冷卻: 0
大招冷卻: 0

Round 5
Player 1 的 Pidgey 使出小招
Player 2 的 Pikachu 使出小招


Player 1 的 Pidgey:
HP: 85/120
energy: 35
小招冷卻: 0
大招冷卻: 0

Player 2 的 Pikachu:
HP: 96/111
energy: 35
小招冷卻: 0
大招冷卻: 0

Round 6
Player 1 的 Pidgey 使出小招
Player 2 的 Pikachu 使出小招


Player 1 的 Pidgey:
HP: 78/120
energy: 42
小招冷卻: 0
大招冷卻: 0

Player 2 的 P